In [1]:
#Imports

import pandas as pd
import string
import re

import nltk
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

import warnings
warnings.filterwarnings("ignore")

In [2]:
#Let's open our excel file.

utterances = pd.read_excel('output_artifacts.xlsx', sheet_name = 'user_queries')
utterances

,text,topic L1,subtopic L2 (if necessary)
0,12.50.00,General,Out_of_scope
1,10 past 3 p.m. I have an appointment on the 22...,Appointment,Appointment
2,15:50 afternoon is good,General,Out_of_scope
3,1st Dec,General,Out_of_scope
4,20 mins after 10 PM,General,Out_of_scope
...,...,...,...
995,I was trying to make an international transfer...,General,Troubleshooting
996,I was trying to make an international transfer...,Application,Application
997,I was trying to top up,Card,Card_refill
998,I was trying to top up my card but it was reve...,General,Troubleshooting


In [3]:
#Now let's tokenize our text data.

tokenized_text = [word_tokenize(sent) for sent in utterances.text.values]
tokens = []

for sent in tokenized_text:
    for word in sent:
         tokens.append(word.lower()) 

print(tokens[:40])

['12.50.00', '10', 'past', '3', 'p.m', '.', 'i', 'have', 'an', 'appointment', 'on', 'the', '22nd', 'may', '15:50', 'afternoon', 'is', 'good', '1st', 'dec', '20', 'mins', 'after', '10', 'pm', '25', 'to', '11', 'morning', 'then', ',', 'bye', '3', 'maybe', '?', '40', 'a', '$', '5600', 'transfer']


In [4]:
#It is time to get rid of the punctuation and stop words.

stop_words = stopwords.words('english')
stop_words.extend(["n't", 'more', 'much', 'really'])

without_punct = [word for word in tokens if word not in list(string.punctuation)]
without_stopwords = [word for word in without_punct if word not in stop_words]

In [5]:
#Now let's apply lemmatization.
lemmatizer = WordNetLemmatizer()
lemma_text = [lemmatizer.lemmatize(w) for w in without_stopwords]

In [6]:
#Let's use TfidfVectorizer.

tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                stop_words = 'english',
                                lowercase = True,
                                token_pattern = r'\b[a-zA-Z]{3,}\b',
                                max_df = 0.5, 
                                min_df = 10)
dtm_tf = tf_vectorizer.fit_transform(lemma_text)
tfidf_vectorizer = TfidfVectorizer(**tf_vectorizer.get_params())
dtm_tfidf = tfidf_vectorizer.fit_transform(lemma_text)

In [7]:
#Now let's create the topic model of the current dataset.

lda_tf = LatentDirichletAllocation(n_components=20, random_state=0)
lda_tf.fit(dtm_tf)
lda_tfidf = LatentDirichletAllocation(n_components=20, random_state=0)
lda_tfidf.fit(dtm_tfidf)
print(lda_tfidf)

LatentDirichletAllocation(n_components=20, random_state=0)


In [9]:
#It is time to create the visualizations of the topic clusters.

from imp import reload
pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer)


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
8     -0.241967 -0.379175       1        1  6.418423
19     0.047852  0.028788       2        1  5.891384
11    -0.254848  0.257706       3        1  5.616407
0     -0.235456  0.099231       4        1  5.547663
15    -0.030970  0.018768       5        1  5.501833
16    -0.007783  0.009582       6        1  5.318515
14     0.040227  0.010088       7        1  5.318515
13     0.041938  0.009461       8        1  5.295600
9      0.023950  0.007741       9        1  5.249771
4      0.013842  0.002821      10        1  5.112282
1      0.044214  0.001791      11        1  4.997709
5      0.054134 -0.001460      12        1  4.814390
12     0.054883 -0.001861      13        1  4.791476
10     0.065269 -0.000517      14        1  4.791476
7      0.030452 -0.007349      15        1  4.722731
3      0.052202 -0.005947      16        1  4.631072
17     0.055516 -0.010954      17        1  4.401925
18     0.075674 -0.013812      18        1  4.012374
2      0.083787 -0.010831      19        1  3.920715
6      0.087085 -0.014070      20        1  3.645738, topic_info=       Term        Freq       Total Category  logprob  loglift
14     card  108.000000  108.000000  Default   30.000  30.0000
1   account   96.000000   96.000000  Default   29.000  29.0000
82     want   86.000000   86.000000  Default   28.000  28.0000
49     need   67.000000   67.000000  Default   27.000  27.0000
45    money   61.000000   61.000000  Default   26.000  26.0000
..      ...         ...         ...      ...      ...      ...
21   credit    0.073049   19.262093  Topic20   -6.993  -2.2632
22  current    0.073049   46.624431  Topic20   -6.993  -3.1471
24      day    0.073049   26.295800  Topic20   -6.993  -2.5744
25    debit    0.073049   58.445584  Topic20   -6.993  -3.3731
26   direct    0.073049   27.499179  Topic20   -6.993  -2.6192

[957 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
0        11  0.963334      abroad
1         3  0.989897     account
2        20  0.937334    activate
3         8  0.960321     address
4         2  0.919660   afternoon
...     ...       ...         ...
84       14  0.946561        week
85       19  0.923437  withdrawal
86        1  0.927907        work
87        7  0.963525        year
88        2  0.952563   yesterday

[89 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[9, 20, 12, 1, 16, 17, 15, 14, 10, 5, 2, 6, 13, 11, 8, 4, 18, 19, 3, 7])